In [ ]:
def Adam_sv_stoc(X,y,learining_rate,beta1,beta2,epsloin,n_iteration):
    m=len(X)
    
    theta0, theta1 = 0, 0
    
    all_loss ,all_theta0,all_theta1,hypothesis_history= [],[],[],[]
    
    v_t0 = 0
    v_t1 = 0
    #m=momentum
    m_t0=0
    m_t1=0
    t=0
   
    for i in range(n_iteration):
        
        #step 2 : h0(x)
        y_hat=theta0+theta1*X
        hypothesis_history.append(y_hat)
        print(f"****************** Iteration {i} ********************\n ")
        print(f"\n h(x):{y_hat}")
      
        error_vector=y_hat-y
        
         #step3 mean square error #mse=loss(j)=error
        loss = np.sum(error_vector**2) / (2 * m)
        all_loss.append(loss)
        all_theta0.append(theta0)
        all_theta1.append(theta1)
        
        # Gradient thetas(1/m)
        #gradient_theta_0 = (1 / m) * np.sum(error_vector)
        #gradient_theta_1 = (1 / m) * np.dot(error_vector, x)
        g_theta0 = sum(y_hat - y) / m
        g_theta1 = sum((y_hat - y) * X) / m
        
        
        # Stopping using gradient
        if np.linalg.norm([g_theta0, g_theta1]) < 0.001:
            break
             # Stopping using loss
        if (i > 0) and abs(all_loss[i-1] - all_loss[i]) < 0.001:
            break
            #updating 
            
        t=t+1
        m_t0= beta1 * m_t0 + (1 - beta1 ) * g_theta0
        m_t1= beta1 * m_t1 + (1 - beta1 ) * g_theta1
        
        v_t0= beta2 * v_t0 + (1 - beta2 ) * g_theta0**2
        v_t1= beta2 * v_t1 + (1 - beta2 ) * g_theta1**2
        
        #bias correction
        m_t0_hat=m_t0 / (1 - beta1**t)
        m_t1_hat=m_t1 / (1 - beta1**t)
        
        v_t0_hat=v_t0 / (1 - beta2**t)
        v_t1_hat=v_t1 / (1 - beta2**t)
        
            
      
        theta0=theta0 - (learining_rate /(np.sqrt(v_t0_hat) + epsloin)) * m_t0_hat
        theta1=theta1- (learining_rate /(np.sqrt(v_t1_hat) + epsloin)) * m_t1_hat
            
            
        
        
    print("\n******* Training Report *******")
    print(f"Gradient Descent converged after {i} epochs")
    print("theta_0_Opt :",theta0)
    print("theta_1_Opt :",theta1)
    print("Cost = ",all_loss[-1])
    print(f"Y_Hat: \n{y_hat}\n")
    print(f"Y: \n{y}")
    
    
    return all_theta0,all_theta1,all_loss,hypothesis_history,theta0,theta1,y_hat

In [ ]:
def Adam_mv_mini_batch(X, Y, learning_rate, max_iters, batch_size, beta1=0.9, beta2=0.999, epsilon=1e-8):
    m = len(Y)
    cost = []
    theta_h = []
    hypothesis_h = []

    # Initialize parameters
    theta = np.zeros((X.shape[1], 1))
    m_theta = np.zeros_like(theta)
    v_theta = np.zeros_like(theta)
    t = 0

    num_batches = int(np.ceil(m / batch_size))

    for i in range(max_iters):
        for j in range(0, m, batch_size):
            
            X_batch = X[j:j+batch_size]
            Y_batch = Y[j:j+batch_size]
            
            theta_h.append(theta.flatten().tolist())
            hypothesis = X_batch.dot(theta)
            hypothesis_h.append(hypothesis)
            error = hypothesis - Y_batch
            loss = np.mean(np.square(error)) / 2
            cost.append(loss)
            gradient = (1 / len(X_batch)) * X_batch.T.dot(error)

            Gradient_Norm = np.linalg.norm(gradient)

            if Gradient_Norm < 0.001:
                break

            if i > 0 and abs(cost[-int(num_batches  + 1)] - cost[-1]) < 0.001:
                break

            t += 1
            m_theta = beta1 * m_theta + (1 - beta1) * gradient
            v_theta = beta2 * v_theta + (1 - beta2) * (gradient ** 2)

            m_hat_theta = m_theta / (1 - beta1 ** t)
            v_hat_theta = v_theta / (1 - beta2 ** t)

            theta -= (learning_rate / (np.sqrt(v_hat_theta) + epsilon)) * m_hat_theta

        print(f"Iteration {i + 1} - Loss: {loss}")

    print("\n******* Training Report *******")
    print(f"Adam_mv converged after {i + 1} iterations")
    print("Optimized parameters:")
    print(theta)
    print("Final Cost:", cost[-1])
    y_final = X @ theta
    return theta, loss, cost,y_final, hypothesis_h

In [ ]:
def adam_stochastic_gradient_descent(X, y, alpha, beta1, beta2, epsilon, epoch, convergence_rate=0.001, norm_value_check=0.001):
    np.random.seed(101)
    alldata = np.column_stack((X, y))
    alldata = shuffle(alldata)
    X = alldata[:, :-1]
    y = alldata[:, -1].reshape(-1, 1)
    m, n = X.shape
    
    theta = np.zeros((n, 1))
    losses = []
    theta_all = []
    
    vt = np.zeros_like(theta)
    mt = np.zeros_like(theta)

    for i in range(epoch):
        for j in range(m):
            # Append theta first
            theta_all.append(theta.copy())

            # Predict
            h = X[j] @ theta

            # Calculate J & Error Vector
            error_vector = h - y[j]
            J = np.sum(error_vector**2) / 2
            losses.append(J)

            # Gradient
            gradient = X[j].reshape(-1, 1) * error_vector
            grad_norm = np.linalg.norm(gradient)

            # Convergence check
            if (grad_norm < norm_value_check):
                print("Converged!")
                return theta, losses, theta_all

            # Momentum equations
            mt = beta1 * mt + (1 - beta1) * gradient

            # RMS equations
            vt = beta2 * vt + (1 - beta2) * (gradient**2)

            # Bias correction
            mt_hat = mt / (1 - beta1**(i + 1))
            vt_hat = vt / (1 - beta2**(i + 1))

            # Adam update theta
            theta = theta - alpha * mt_hat / (np.sqrt(vt_hat) + epsilon)

        if ((i > 1) and (abs(losses[- 1] - losses[-(m+1)]) < convergence_rate)):
            break
            
        # Print information for each epoch
        print(f"****************** Epoch {i} ********************")
        print(f"h(x): {h.flatten()}")
        print(f"Error Vector: {error_vector.flatten()}")
        print(f"J = {J}")
        print(f"Gradient Vector:\n{gradient}")
        print(f"Gradient Vector Norm: {grad_norm}")
        print(f"Theta_new : {theta.flatten()}\n")

    print("Did not converge within the specified number of epochs.")
    return theta, losses, theta_all
